In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import missingno as msno
import nltk
import string
import re
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

df = pd.read_csv('transformed_data.csv')
df=df.dropna(axis=0,inplace=False)
clean_text_df = pd.read_csv('clean_text.csv')
clean_text_df=clean_text_df.dropna(axis=0,inplace=False)
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

#df['body_len'] = df['text'].apply(lambda x: len(x) - x.count(" "))
#df['punct%'] = df['text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df.head()

,links,text,value,body punct%,title punct%,punct%,try,count_punct,count_ws,try2,body_len,twitter
0,"Not All COVID-19 Aid Is Spent. But Schools, Ci...",An analysis by J.P. Morgan found that nationwi...,T,-600.0,-300.0,-600.0,-600.0,1.220285,1.709976,0.843162,2.643126,0
1,"Biden calls for national unity and healing, as...","As the moment of silence concluded, Biden, a d...",T,-400.0,-200.0,-400.0,-400.0,1.166529,1.721190,0.809683,2.705662,0
2,Capitol security officials in charge on Jan. 6...,The Pentagon has contested the timeline Sund o...,T,-800.0,-200.0,-800.0,-800.0,1.259921,1.912931,0.868404,3.052454,0
3,This Congress is the most diverse ever. But Hi...,Heather Caygle contributed to this report.,T,-100.0,-200.0,-100.0,-100.0,1.000000,1.307660,0.704424,2.058924,0
4,"With Or Without Trump, Republicans Will Likely...","Reagan also won primaries in Texas, Georgia an...",T,-400.0,-100.0,-400.0,-400.0,1.166529,1.772394,0.809806,2.799663,0


In [3]:

del df['try']
df['punct%']=df['try2']
del df['try2']
df.head()

,links,text,value,body punct%,title punct%,punct%,count_punct,count_ws,body_len,twitter
0,"Not All COVID-19 Aid Is Spent. But Schools, Ci...",An analysis by J.P. Morgan found that nationwi...,T,-600.0,-300.0,0.843162,1.220285,1.709976,2.643126,0
1,"Biden calls for national unity and healing, as...","As the moment of silence concluded, Biden, a d...",T,-400.0,-200.0,0.809683,1.166529,1.721190,2.705662,0
2,Capitol security officials in charge on Jan. 6...,The Pentagon has contested the timeline Sund o...,T,-800.0,-200.0,0.868404,1.259921,1.912931,3.052454,0
3,This Congress is the most diverse ever. But Hi...,Heather Caygle contributed to this report.,T,-100.0,-200.0,0.704424,1.000000,1.307660,2.058924,0
4,"With Or Without Trump, Republicans Will Likely...","Reagan also won primaries in Texas, Georgia an...",T,-400.0,-100.0,0.809806,1.166529,1.772394,2.799663,0


In [4]:
#due to some majore issues with size, I will be reducing the df size
#after trial and error, 25% was found to be operable, 50%+ crashed the computer even with extra steps
#def dovide(x):
    #n=25
    #m=75
    #fir =x.tail(int(len(x)*(m/100)))
    #x = x.head(int(len(x)*(n/100)))
    #return x,fir

In [5]:
#df,df2= dovide(df)
df.shape

(2319, 10)

In [6]:
clean_text_df.head()

,title,text,value
0,"['judge', 'rip', 'capitol', 'rioter', 'trump',...","['chrestman', 'face', 'series', 'felony', 'cha...",T
1,"['republican', 'rift', 'go', 'far', 'deeper', ...","['higher', 'normal', 'number', 'ten', 'thousan...",T
2,"['covid19', 'aid', 'spent', 'school', 'city', ...","['congress', 'already', 'appropriated', '150',...",T
3,"['covid19', 'aid', 'spent', 'school', 'city', ...","['hutchinson', 'vice', 'chair', 'national', 'g...",T
4,"['congress', 'diverse', 'ever', 'hill', 'staff...","['latest', 'increase', 'diversity', 'due', 'la...",T


In [7]:
sum(df['twitter'])

0

In [8]:
from sklearn.model_selection import train_test_split

# splitting and testing the body text

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df[['text', 'body_len', 'punct%']], df['value'], test_size=0.45)

In [10]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])

tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)


X_train_vect.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,4563,4564,4565,4566,4567,4568,4569,4570,4571,4572
0,3.052454,0.843726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.167803,0.857068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.338943,0.809463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.759459,0.827929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.997527,0.843572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X_train_vect.shape

(1275, 4575)

# Model Eval

In [12]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [13]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=False, average='weighted')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 0.285 / Predict time: 0.083 ---- Precision: 0.97 / Recall: 0.969 / Accuracy: 0.969


C:\Users\alexr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1267: UserWarning: Note that pos_label (set to False) is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [ ]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=False, average='weighted')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

In [ ]:
#So we see the values are close, but the gradient is a better option, unles the time is an issue